In [1]:
import os
import numpy as np
import h5py
import cv2
import tifffile

def list_files(folder, extension='.tiff'):
    filenames = os.listdir(folder)
    filepaths = [os.path.join(folder, filename) for filename in filenames]
    return filepaths

def load_image(file_path):
    tiff_data = tifffile.imread(file_path)
    np_array = np.array(tiff_data)
    return np_array

def binarize(image):
    _, binary = cv2.threshold(image, 0, 1, cv2.THRESH_BINARY)
    return binary

def create_crops(image, label, size=256):
    crops = []
    labels = []
    h, w, _ = image.shape
    for i in range(0, h - size + 1, size):
        for j in range(0, w - size + 1, size):
            crop_img = image[i:i+size, j:j+size]
            crop_label = label[i:i+size, j:j+size]
            if np.mean(crop_label) > 0.01 and 0 < np.mean(crop_img) < 255:  # Checking if label is not 99% black
                crops.append(crop_img)
                labels.append(crop_label)
    return crops, labels

def main():
    output_image_dir = "/kaggle/working/images"
    output_label_dir = "/kaggle/working/labels"
    if not os.path.exists(output_image_dir):
        os.mkdir(output_image_dir)
    if not os.path.exists(output_label_dir):
        os.mkdir(output_label_dir)

        
    folders = ['/kaggle/input/massachusetts-roads-dataset/tiff/train', '/kaggle/input/massachusetts-roads-dataset/tiff/test', '/kaggle/input/massachusetts-roads-dataset/tiff/val']
#     h5_file = h5py.File('dataset.h5', 'w')
    count = 0
    for folder in folders:
        image_folder = f'{folder}/'
        label_folder = f'{folder}_labels/'

        label_paths = list_files(label_folder)
#         print(label_paths)

        for lbl_path in label_paths:
            if not os.path.exists(os.path.join(image_folder, os.path.basename(lbl_path)+'f')):
                print("doesnt")
                continue
            image = load_image(os.path.join(image_folder, os.path.basename(lbl_path)+'f'))
            label = load_image(lbl_path)

            crops, labels = create_crops(image, label)

            for idx, (crop_img, crop_label) in enumerate(zip(crops, labels)):
                crop_img = crop_img
                crop_label = binarize(crop_label)

                cv2.imwrite(os.path.join(output_image_dir, f"{count}.png"), crop_img)
                cv2.imwrite(os.path.join(output_label_dir, f"{count}.png"), crop_label)
#                 file_name = f'{folder}_{idx}'
#                 h5_file.create_dataset(f'images/{count}', data=crop_img)
#                 h5_file.create_dataset(f'labels/{count}', data=crop_label)
                count = count+1

#     h5_file.close()


main()


In [2]:
# old_path = "/kaggle/input/massachusetts-roads-dataset/tiff/train"
# paths = {"train":["/kaggle/input/massachusetts-roads-dataset/tiff/train", "/kaggle/input/massachusetts-roads-dataset/tiff/train_labels"],
#         "val":["/kaggle/input/massachusetts-roads-dataset/tiff/val", "/kaggle/input/massachusetts-roads-dataset/tiff/val_labels"],
#         "test":["/kaggle/input/massachusetts-roads-dataset/tiff/test","/kaggle/input/massachusetts-roads-dataset/tiff/test_labels"]}
old_path = "/kaggle/input/massachusetts-roads-dataset/tiff/train"
paths = {"train":["/kaggle/working/images", "/kaggle/working/labels"]}

In [3]:
import os
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

#New generator with rotation and shear where interpolation that comes with rotation and shear are thresholded in masks. 
#This gives a binary mask rather than a mask with interpolated values. 
seed=24
batch_size= 16
from tensorflow.keras.preprocessing.image import ImageDataGenerator




# img_data_gen_args = dict(rescale = 1/255.,
#                          rotation_range=90,
#                       width_shift_range=0.3,
#                       height_shift_range=0.3,
#                       shear_range=0.5,
#                       zoom_range=0.3,
#                       horizontal_flip=True,
#                       vertical_flip=True,
#                       fill_mode='reflect')

# mask_data_gen_args = dict(rescale = 1/255.,  #Original pixel values are 0 and 255. So rescaling to 0 to 1
#                         rotation_range=90,
#                       width_shift_range=0.3,
#                       height_shift_range=0.3,
#                       shear_range=0.5,
#                       zoom_range=0.3,
#                       horizontal_flip=True,
#                       vertical_flip=True,
#                       fill_mode='reflect',
#                       preprocessing_function = lambda x: np.where(x>0, 1, 0).astype(x.dtype)) #Binarize the output again. 
img_data_gen_args = dict(rescale = 1/255.)
mask_data_gen_args = dict()

#If You need to resize images then add this to the flow_from_directory parameters 
#target_size=(150, 150), #Or whatever the size is for your network

image_data_generator = ImageDataGenerator(**img_data_gen_args)
image_generator = image_data_generator.flow_from_directory(paths["train"][0], 
                                                           seed=seed, 
                                                           batch_size=batch_size,
                                                           classes = ["."],
                                                           class_mode=None
                                                           )  #Very important to set this otherwise it returns multiple numpy arrays 
                                                                            #thinking class mode is binary.

mask_data_generator = ImageDataGenerator(**mask_data_gen_args)
mask_generator = mask_data_generator.flow_from_directory(paths["train"][1], 
                                                        classes = ["."],
                                                        seed=seed, 
                                                        batch_size=batch_size,
                                                        color_mode = 'grayscale',   #Read masks in grayscale
                                                         class_mode=None
                                                         )


# valid_img_generator = image_data_generator.flow_from_directory(paths["val"][0],
#                                                                seed=seed, 
#                                                                batch_size=batch_size, 
#                                                                classes = ["."],
#                                                                class_mode=None
#                                                                ) #Default batch size 32, if not specified here
# valid_mask_generator = mask_data_generator.flow_from_directory(paths["val"][1], 
#                                                                classes = ["."],
#                                                                seed=seed, 
#                                                                batch_size=batch_size, 
#                                                                color_mode = 'grayscale',   #Read masks in grayscale
#                                                                class_mode=None
#                                                               )  #Default batch size 32, if not specified here


train_generator = zip(image_generator, mask_generator)
# val_generator = zip(valid_img_generator, valid_mask_generator)

2024-04-14 09:07:36.202855: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-14 09:07:36.202956: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-14 09:07:36.326726: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Found 20975 images belonging to 1 classes.
Found 20975 images belonging to 1 classes.


In [4]:
import warnings
warnings.filterwarnings('ignore')

import os
import cv2
import numpy as np
import tensorflow as tf
import pandas as pd
from keras.models import Model, load_model
from skimage.morphology import label
import pickle
import tensorflow.keras.backend as K

from matplotlib import pyplot as plt
from tqdm import tqdm_notebook
import random
from skimage.io import imread, imshow, imread_collection, concatenate_images
from matplotlib import pyplot as plt
import h5py

seed = 56

In [5]:
# IMAGE_HEIGHT = IMAGE_WIDTH = 256
# NUM_CHANNELS = 3
# # image_file = './Data/road_images.h5py'
# # mask_file = './Data/road_masks.h5py'
# image_file = "/kaggle/working/dataset.h5"

# def load_group_to_numpy(group):
#     data_list = []
    
#     # Iterate over the keys (dataset names) in the group
#     for key in group.keys():
#         dataset = group.get(key)
        
#         # Check if the object is a dataset
#         if isinstance(dataset, h5py.Dataset):
#             data = np.array(dataset)  # Load dataset into a NumPy array
#             data_list.append(data)
    
#     return np.array(data_list)

In [6]:
# hfile = h5py.File(image_file, 'r')
# n1 = hfile.get('images')
# images = load_group_to_numpy(n1)
# images = np.array(images)
# print(images.shape)
# hfile.close()

In [7]:
# hfile = h5py.File(image_file, 'r')
# n1 = hfile.get('labels')
# masks = load_group_to_numpy(n1)
# masks = np.array(masks)
# print(masks.shape)
# print("Unique elements in the train mask:", np.unique(masks))
# hfile.close()
# # print(masks[0])


In [8]:
# print(images[0])

In [9]:
# plt.figure(figsize=(20,16))
# x, y = 5,4
# for i in range(y):  
#     for j in range(x):
#         plt.subplot(y*2, x, i*2*x+j+1)
#         pos = i*120 + j*10
#         plt.imshow(images[pos])
#         plt.title('Sat img #{}'.format(pos))
#         plt.axis('off')
#         plt.subplot(y*2, x, (i*2+1)*x+j+1)
           
#         #We display the associated mask we just generated above with the training image
#         plt.imshow(masks[pos])
#         plt.title('Mask #{}'.format(pos))
#         plt.axis('off')
        
# plt.show()

In [10]:
# masks = np.expand_dims(masks, -1)

In [11]:
# print(masks.shape)

In [12]:

# print(masks.shape)
# print(images.shape)

In [13]:
def iou_coef(y_true, y_pred, smooth=1e-6):
    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection
    
    return (intersection + smooth) / (union + smooth)

def dice_coef(y_true, y_pred, smooth=1e-6):
    intersection = tf.reduce_sum(y_true * y_pred)
    total = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred)
    
    return (2. * intersection + smooth) / (total + smooth)

def soft_dice_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)


In [14]:
# from sklearn.model_selection import train_test_split
# images = images.astype(np.uint8)
# masks = masks.astype(np.uint8)
# train_images, test_images, train_masks, test_masks = train_test_split(images, masks, test_size=0.2, random_state=seed)
# del images, masks
# print("TRAIN SET")
# print(train_images.shape)
# print(train_masks.shape)
# print("TEST SET")
# print(test_images.shape)
# print(test_masks.shape)

In [15]:
IMAGE_HEIGHT = IMAGE_WIDTH = 256

In [16]:
from keras.models import Model, load_model
import tensorflow as tf
from keras.layers import Input
from keras.layers import Dropout, Lambda
from keras.layers import Conv2D, Conv2DTranspose
from keras.layers import MaxPooling2D
from keras.layers import concatenate
from keras import optimizers
from keras.layers import BatchNormalization
from tensorflow.keras.metrics import MeanIoU
import keras

In [17]:

inputs = Input((IMAGE_HEIGHT, IMAGE_WIDTH, 3))
# s = Lambda(lambda x: x / 255) (inputs)

conv1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (inputs)
conv1 = BatchNormalization() (conv1)
conv1 = Dropout(0.1) (conv1)
conv1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (conv1)
conv1 = BatchNormalization() (conv1)
pooling1 = MaxPooling2D((2, 2)) (conv1)

conv2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (pooling1)
conv2 = BatchNormalization() (conv2)
conv2 = Dropout(0.1) (conv2)
conv2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (conv2)
conv2 = BatchNormalization() (conv2)
pooling2 = MaxPooling2D((2, 2)) (conv2)

conv3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (pooling2)
conv3 = BatchNormalization() (conv3)
conv3 = Dropout(0.2) (conv3)
conv3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (conv3)
conv3 = BatchNormalization() (conv3)
pooling3 = MaxPooling2D((2, 2)) (conv3)

conv4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (pooling3)
conv4 = BatchNormalization() (conv4)
conv4 = Dropout(0.2) (conv4)
conv4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (conv4)
conv4 = BatchNormalization() (conv4)
pooling4 = MaxPooling2D(pool_size=(2, 2)) (conv4)

conv5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (pooling4)
conv5 = BatchNormalization() (conv5)
conv5 = Dropout(0.3) (conv5)
conv5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (conv5)
conv5 = BatchNormalization() (conv5)


upsample6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (conv5)
upsample6 = concatenate([upsample6, conv4])
conv6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (upsample6)
conv6 = BatchNormalization() (conv6)
conv6 = Dropout(0.2) (conv6)
conv6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (conv6)
conv6 = BatchNormalization() (conv6)

upsample7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (conv6)
upsample7 = concatenate([upsample7, conv3])
conv7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (upsample7)
conv7 = BatchNormalization() (conv7)
conv7 = Dropout(0.2) (conv7)
conv7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (conv7)
conv7 = BatchNormalization() (conv7)

upsample8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (conv7)
upsample8 = concatenate([upsample8, conv2])
conv8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (upsample8)
conv8 = BatchNormalization() (conv8)
conv8 = Dropout(0.1) (conv8)
conv8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (conv8)
conv8 = BatchNormalization() (conv8)

upsample9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (conv8)
upsample9 = concatenate([upsample9, conv1], axis=3)
conv9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (upsample9)
conv9 = BatchNormalization() (conv9)
conv9 = Dropout(0.1) (conv9)
conv9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (conv9)
conv9 = BatchNormalization() (conv9)

outputs = Conv2D(1, (1, 1), activation='sigmoid') (conv9)

model = Model(inputs=[inputs], outputs=[outputs])
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 256, 256,  │        448 │ input_layer[0][0] │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 256, 256,  │         64 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 256, 256,  │          0 │ batch_normalizat… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 256, 256,  │      2,320 │ dropout[0][0]     │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256, 256,  │         64 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 128, 128,  │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 128, 128,  │      4,640 │ max_pooling2d[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        128 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 128, 128,  │          0 │ batch_normalizat… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 128, 128,  │      9,248 │ dropout_1[0][0]   │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        128 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 64, 64,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 64, 64,    │     18,496 │ max_pooling2d_1[… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64,    │        256 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 64, 64,    │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 64, 64,    │     36,928 │ dropout_2[0][0] 

 Total params: 1,946,993 (7.43 MB)

 Trainable params: 1,944,049 (7.42 MB)

 Non-trainable params: 2,944 (11.50 KB)

In [18]:
EPOCHS = 100
LEARNING_RATE = 0.0001
BATCH_SIZE = 16

In [19]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from datetime import datetime

In [20]:
model_path = "/kaggle/working/road_mapper_2.keras"
checkpointer = ModelCheckpoint(model_path,
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystopper = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 5,
                          verbose = 1,
                          restore_best_weights = True)

lr_reducer = ReduceLROnPlateau(monitor='val_loss',
                               factor=0.1,
                               patience=4,
                               verbose=1,
                               epsilon=1e-4)

In [21]:
opt = keras.optimizers.Adam(LEARNING_RATE)
model.compile(
      optimizer=opt,
      loss=soft_dice_loss,
      metrics=[iou_coef])

In [22]:
def create_generator(zipped):
    for (img, mask) in zipped:
        yield (img, mask)

In [ ]:

history = model.fit(create_generator(train_generator), 
#                     validation_data = create_generator(val_generator),
#                     validation_split = 0.1,
    epochs=EPOCHS,
    steps_per_epoch = len(os.listdir(paths["train"][0])) //BATCH_SIZE,
    batch_size = BATCH_SIZE,
    callbacks = [checkpointer, earlystopper, lr_reducer],
#                     validation_steps = len(os.listdir(paths["val"][0])) // BATCH_SIZE
)

Epoch 1/100


2024-04-14 09:08:09.719862: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 65536: 3.41604, expected 2.96419
2024-04-14 09:08:09.720261: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 131072: 2.39036, expected 1.63582
2024-04-14 09:08:09.720300: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 131073: 3.72286, expected 2.96832
2024-04-14 09:08:09.720315: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 131074: 2.60526, expected 1.85073
2024-04-14 09:08:09.720328: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 131075: 4.48829, expected 3.73375
2024-04-14 09:08:09.720341: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 131076: 4.84788, expected 4.09334
2024-04-14 09:08:09.720349: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 131077: 4.35289, expected 3.59836
2024-04-14 09:08:09.7

   2/1310 ━━━━━━━━━━━━━━━━━━━━ 2:00 92ms/step - iou_coef: 0.0763 - loss: 0.8583   

I0000 00:00:1713085712.794110      99 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1310/1310 ━━━━━━━━━━━━━━━━━━━━ 152s 83ms/step - iou_coef: 0.1717 - loss: 0.7140 - learning_rate: 1.0000e-04
Epoch 2/100


2024-04-14 09:10:29.323400: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 0: 2.52528, expected 2.00293
2024-04-14 09:10:29.323460: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 1: 3.9192, expected 3.39686
2024-04-14 09:10:29.323470: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 2: 4.05446, expected 3.53212
2024-04-14 09:10:29.323478: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 8: 3.69228, expected 3.16994
2024-04-14 09:10:29.323485: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 9: 4.0809, expected 3.55856
2024-04-14 09:10:29.323493: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 21: 4.03202, expected 3.50968
2024-04-14 09:10:29.323501: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 29: 4.00203, expected 3.47969
2024-04-14 09:10:29.323509: E external/local_xla/xla/se

1310/1310 ━━━━━━━━━━━━━━━━━━━━ 138s 83ms/step - iou_coef: 0.4065 - loss: 0.4245 - learning_rate: 1.0000e-04
Epoch 3/100
1310/1310 ━━━━━━━━━━━━━━━━━━━━ 109s 83ms/step - iou_coef: 0.5006 - loss: 0.3345 - learning_rate: 1.0000e-04
Epoch 4/100
1310/1310 ━━━━━━━━━━━━━━━━━━━━ 109s 83ms/step - iou_coef: 0.5286 - loss: 0.3097 - learning_rate: 1.0000e-04
Epoch 5/100
1310/1310 ━━━━━━━━━━━━━━━━━━━━ 108s 83ms/step - iou_coef: 0.5420 - loss: 0.2987 - learning_rate: 1.0000e-04
Epoch 6/100
1310/1310 ━━━━━━━━━━━━━━━━━━━━ 108s 83ms/step - iou_coef: 0.5538 - loss: 0.2886 - learning_rate: 1.0000e-04
Epoch 7/100
1310/1310 ━━━━━━━━━━━━━━━━━━━━ 109s 83ms/step - iou_coef: 0.5554 - loss: 0.2875 - learning_rate: 1.0000e-04
Epoch 8/100
1310/1310 ━━━━━━━━━━━━━━━━━━━━ 109s 83ms/step - iou_coef: 0.5623 - loss: 0.2818 - learning_rate: 1.0000e-04
Epoch 9/100
1310/1310 ━━━━━━━━━━━━━━━━━━━━ 109s 83ms/step - iou_coef: 0.5677 - loss: 0.2773 - learning_rate: 1.0000e-04
Epoch 10/100
1310/1310 ━━━━━━━━━━━━━━━━━━━━ 109s 83m

In [ ]:
model.save("kaggle/working/road_mapper_final.keras")

In [ ]:
model = load_model("/kaggle/working/road_mapper_final.keras", custom_objects={'soft_dice_loss': soft_dice_loss, 'iou_coef': iou_coef})


In [ ]:
model.evaluate(test_images, test_masks)


In [ ]:
predictions = model.predict(test_images, verbose=1)

In [ ]:
thresh_val = 0.1
predicton_threshold = (predictions > thresh_val).astype(np.uint8)

In [ ]:
ix = random.randint(0, len(predictions))
num_samples = 10

f = plt.figure(figsize = (15, 25))
for i in range(1, num_samples*4, 4):
  ix = random.randint(0, len(predictions))

  f.add_subplot(num_samples, 4, i)
  imshow(test_images[ix][:,:,0])
  plt.title("Image")
  plt.axis('off')

  f.add_subplot(num_samples, 4, i+1)
  imshow(np.squeeze(test_masks[ix][:,:,0]))
  plt.title("Groud Truth")
  plt.axis('off')

  f.add_subplot(num_samples, 4, i+2)
  imshow(np.squeeze(predictions[ix][:,:,0]))
  plt.title("Prediction")
  plt.axis('off')

  f.add_subplot(num_samples, 4, i+3)
  imshow(np.squeeze(predicton_threshold[ix][:,:,0]))
  plt.title("thresholded at {}".format(thresh_val))
  plt.axis('off')

plt.show()